In [1]:
import pandas as pd
import re
import os
import warnings
warnings.filterwarnings('ignore')

#pd.set_option('display.max_rows', 500)

# DATA REVIEW / STEP 1

Veri Seti okunuyor ve ilgili alandaki işler farklı bir excel dosyasına kayıt ediliyor.

In [2]:
raw_data = pd.read_excel(os.path.abspath('..')+"/Data/110K_IlanDatasi_23012020_Canli.xlsx")

In [3]:
raw_data.head()

,Unnamed: 0,jobrefno,EgitimDurumu,Tecrube,maxTecrube,iller,PozisyonAdi,metin
0,0,2451579,"YM,UO,UM,MO,MM,DO,DM",0,10,Eskişehir,Genel Başvuru,<p>Genel başvuru&nbsp;amaçlı bu ilan firmamızd...
1,1,2451577,"KM,LM,YM,UM,MM",99,99,İstanbul(Avr.),Uluslararası Hasta İlişkileri Uzmanı,<ul><li><u><strong>Çok iyi düzeyde&nbsp;İTALYA...
2,2,2451576,KM,99,99,İstanbul(Avr.),Dilme Makinesi Operatörü,<p>DİSİPLİNLİ TEMİZLİĞE DÜZENE ÖNEM VEREN TECR...
3,3,2451575,"LM,YM,UM,MM",99,99,İstanbul(Avr.),Uluslararası Hasta İlişkileri Uzmanı,<ul><li><u><strong>Çok iyi düzeyde&nbsp;ALMANC...
4,4,2451574,LM,99,99,İstanbul(Avr.),Modelist,<p></p><li>İhracat firmalarında dokuma ve örme...


## GET TEXT AND POSITION

In [4]:
Jobs_positions_text_df= raw_data[["PozisyonAdi","metin"]]

In [5]:
Jobs_positions_text_df.head()

,PozisyonAdi,metin
0,Genel Başvuru,<p>Genel başvuru&nbsp;amaçlı bu ilan firmamızd...
1,Uluslararası Hasta İlişkileri Uzmanı,<ul><li><u><strong>Çok iyi düzeyde&nbsp;İTALYA...
2,Dilme Makinesi Operatörü,<p>DİSİPLİNLİ TEMİZLİĞE DÜZENE ÖNEM VEREN TECR...
3,Uluslararası Hasta İlişkileri Uzmanı,<ul><li><u><strong>Çok iyi düzeyde&nbsp;ALMANC...
4,Modelist,<p></p><li>İhracat firmalarında dokuma ve örme...


## DATA DESCRIPTION

In [6]:
Jobs_positions_text_df["PozisyonAdi"].value_counts().head(20)

Satış Danışmanı                     3261
Mağaza Müdürü                       1348
Muhasebe Elemanı                    1136
Satış Temsilcisi                    1083
Muhasebe Uzmanı                     1040
Satış Elemanı                        886
Müşteri Temsilcisi                   871
Yönetici Asistanı                    857
Genel Başvuru                        848
Mağaza Satış Danışmanı               773
Muhasebe Sorumlusu                   751
Sekreter                             739
Ön Muhasebe Elemanı                  737
Çağrı Merkezi Müşteri Temsilcisi     644
İnsan Kaynakları Uzmanı              624
Resepsiyonist                        599
Makine Mühendisi                     593
Yazılım Uzmanı                       584
Mağaza Müdür Yardımcısı              580
Satış Uzmanı                         525
Name: PozisyonAdi, dtype: int64

In [7]:
Jobs_positions_text_df["PozisyonAdi"].value_counts().count()

6904

## SELECT DATA

In [8]:
Jobs_positions_text_df["PozisyonAdi"]=Jobs_positions_text_df["PozisyonAdi"].str.lower()

In [9]:
position_name= "yazılım uzmanı"
Jobs_positions_text_df[Jobs_positions_text_df["PozisyonAdi"].str.contains(position_name)].count()

PozisyonAdi    1123
metin          1123
dtype: int64

In [10]:
yazilim_jobs_df = Jobs_positions_text_df[Jobs_positions_text_df["PozisyonAdi"].str.contains(position_name)]

In [11]:
yazilim_jobs_df[Jobs_positions_text_df["PozisyonAdi"].str.contains(position_name)][["PozisyonAdi"]].value_counts()
#.sort_index(ascending=True)

PozisyonAdi               
yazılım uzmanı                584
java yazılım uzmanı            97
web yazılım uzmanı             93
mobil yazılım uzmanı           69
arayüz yazılım uzmanı          64
kıdemli yazılım uzmanı         60
asp.net yazılım uzmanı         58
php yazılım uzmanı             58
oyun yazılım uzmanı            14
oracle yazılım uzmanı          12
sql yazılım uzmanı             10
gömülü yazılım uzmanı           3
dynamics ax yazılım uzmanı      1
dtype: int64

In [12]:
yazilim_jobs_df.to_excel("yazilim_uzmani_jobs.xlsx")  

# CLENING IN JOBS / STEP 2

HTML taglerinin, gereksiz karakterlerin, fazla boşlukların temizlenmesi. Madde işaretlerinin belirlenmesi ve farklı bir şekilde kaydedilmesi.

In [13]:
raw_data = pd.read_excel("yazilim_uzmani_jobs.xlsx")
raw_data.head()

,Unnamed: 0,PozisyonAdi,metin
0,134,yazılım uzmanı,"<p>Dünyada 150 ülkede, 27 farklı dil ile yakla..."
1,137,yazılım uzmanı,"<p class=""x_x_x_MsoNormal""><strong><u><span>GE..."
2,270,oyun yazılım uzmanı,<p>Oyun geliştirmeyi ve oynamayı hayat felsefe...
3,586,yazılım uzmanı,<p>Şirketimizde&nbsp;görevlendirilmek üzere aş...
4,624,yazılım uzmanı,<ul> <li><span>Yazılım geliştirme projelerind...


## CLEANING HTML TAGS AND UNNECESSARY CHARACTERS

In [14]:
data = raw_data.copy()

### TO LOWER

In [15]:
data["metin"] = data["metin"].str.lower()

### \<li> TAG DETECT FUNCTION

In [16]:
def change_li(text):
    clean = re.compile('(<li>)')
    return re.sub(clean, ' BLLTX ', text)
#<li> tag'ini farklı bir şekilde kaydetmemizi sağlar. Bu sayede aynı cümle içinde geçen özellikleri kaybetmemiş oluruz. 

### HTML TAGS REMOVE FUNCTION

In [17]:
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, ' ', text)
#html tag'lerini temizlememizi sağlar. 

### CLEANING UNNECESSARY CHARACTERS

In [18]:
data["metin"]=data["metin"].replace(to_replace=["&nbsp;"],value=" ",regex=True,inplace=False)
data.head()

,Unnamed: 0,PozisyonAdi,metin
0,134,yazılım uzmanı,"<p>dünyada 150 ülkede, 27 farklı dil ile yakla..."
1,137,yazılım uzmanı,"<p class=""x_x_x_msonormal""><strong><u><span>ge..."
2,270,oyun yazılım uzmanı,<p>oyun geliştirmeyi ve oynamayı hayat felsefe...
3,586,yazılım uzmanı,<p>şirketimizde görevlendirilmek üzere aşağıda...
4,624,yazılım uzmanı,<ul> <li><span>yazılım geliştirme projelerind...


### CLEANING HTML TAGS 

In [19]:
data["metin"]=data['metin'].apply(lambda text : change_li(text))
data.head()

,Unnamed: 0,PozisyonAdi,metin
0,134,yazılım uzmanı,"<p>dünyada 150 ülkede, 27 farklı dil ile yakla..."
1,137,yazılım uzmanı,"<p class=""x_x_x_msonormal""><strong><u><span>ge..."
2,270,oyun yazılım uzmanı,<p>oyun geliştirmeyi ve oynamayı hayat felsefe...
3,586,yazılım uzmanı,<p>şirketimizde görevlendirilmek üzere aşağıda...
4,624,yazılım uzmanı,<ul> BLLTX <span>yazılım geliştirme projeler...


In [20]:
data["metin"]=data['metin'].apply(lambda metin : remove_html_tags(metin))
data.head()

,Unnamed: 0,PozisyonAdi,metin
0,134,yazılım uzmanı,"dünyada 150 ülkede, 27 farklı dil ile yaklaşı..."
1,137,yazılım uzmanı,genel ni̇teli̇kler BLLTX üniversit...
2,270,oyun yazılım uzmanı,oyun geliştirmeyi ve oynamayı hayat felsefesi...
3,586,yazılım uzmanı,şirketimizde görevlendirilmek üzere aşağıdaki...
4,624,yazılım uzmanı,BLLTX yazılım geliştirme projelerinde akt...


## REMOVE TOO MUCH SPACE

In [21]:
data["metin"]=data['metin'].apply(lambda text : re.sub(' +', ' ', text))

## CHANGE BULLET SIGN

In [22]:
data["metin"]=data["metin"].replace(to_replace="·|•|\*",value=" BLLTX ",regex=True,inplace=False)

In [23]:
data["metin"][45]

' 1. php diline hakim, 2. mysql veritabanı n da deneyimli 3. web uygulamaları geliştirme konusunda deneyimli (html5, jquery, ajax, css3 tecrübesi bulunan) 4. i̇leri seviyede wordpress eklenti ve temalara hakim , 5. linux server ile çalışabilen , 6. yaptığı işlerle ilgili referans gösterebilen, 7. teknik dokümantasyonu takip edebilecek seviyede i̇ngilizce bilen 8. erkek adaylar için askerliğini tamamlamış veya 3 yıl tecilli 9. benzer pozisyonda en az 1 yıl deneyimli, '

# MASKING FOR SPECIAL TERMS / STEP 3

Özel karakter içeren skiller ve diğer başka skiller için maskeleme işleminin yapılması

In [24]:
data_without_special_terms = data.copy()

## CREATE DICTIONARY

Special Term'leri okuyup iki yönlü bir sözlük oluşturuluyor.

In [25]:
special_terms_dictionary = {}
reversed_special_terms_dictionary = {}
with open(os.path.abspath('..')+"\Data\_special_terms.txt",encoding="utf8") as file:
    line_counter=1
    for line in file:
        line=line.lower()
        #if("+" in line):
        #    line=line.replace("+","\+",1) #Regex'ten dolayı \ ekleme işlemi
        special_terms_dictionary[line.strip("\n")] = "SPCL"+str(line_counter)+"SPCL"
        reversed_special_terms_dictionary["SPCL"+str(line_counter)+"SPCL"] = line.strip()
        line_counter +=1

In [26]:
special_terms_dictionary

{'swift': 'SPCL1SPCL',
 'kotlin': 'SPCL2SPCL',
 'android': 'SPCL3SPCL',
 'backend': 'SPCL4SPCL',
 'back end': 'SPCL5SPCL',
 'gömülü sistem': 'SPCL6SPCL',
 'iot': 'SPCL7SPCL',
 'embedded systems': 'SPCL8SPCL',
 'makine öğrenmesi': 'SPCL9SPCL',
 'machine learning': 'SPCL10SPCL',
 'önyüz': 'SPCL11SPCL',
 'ön yüz': 'SPCL12SPCL',
 'frontend': 'SPCL13SPCL',
 'front end': 'SPCL14SPCL',
 'fullstack': 'SPCL15SPCL',
 'full stack': 'SPCL16SPCL',
 'ios': 'SPCL17SPCL',
 'perl': 'SPCL18SPCL',
 'rust': 'SPCL19SPCL',
 'html': 'SPCL20SPCL',
 'unity': 'SPCL21SPCL',
 'unity3d': 'SPCL22SPCL',
 'python': 'SPCL23SPCL',
 'java': 'SPCL24SPCL',
 'javascript': 'SPCL25SPCL',
 ' r ': 'SPCL26SPCL',
 'php': 'SPCL27SPCL',
 'visual basic': 'SPCL28SPCL',
 'go': 'SPCL29SPCL',
 'ruby': 'SPCL30SPCL',
 'asp. net core': 'SPCL31SPCL',
 'asp. net': 'SPCL32SPCL',
 'ado.net': 'SPCL33SPCL',
 'asp.net ajax': 'SPCL34SPCL',
 'asp.net ': 'SPCL35SPCL',
 'mvc': 'SPCL36SPCL',
 ' c ': 'SPCL37SPCL',
 ' sap ': 'SPCL38SPCL',
 ' gi̇t ': 'S

## MASKING

In [27]:
for key,val in special_terms_dictionary.items():
    data_without_special_terms["metin"] = data_without_special_terms["metin"].apply(lambda text: text.replace(key," "+val+" "))    

In [28]:
data_without_special_terms["metin"][45]

' 1.  SPCL27SPCL  diline hakim, 2. mysql veritabanı n da deneyimli 3. web uygulamaları geliştirme konusunda deneyimli ( SPCL20SPCL 5, jquery, ajax, css3 tecrübesi bulunan) 4. i̇leri seviyede wordpress eklenti ve temalara hakim , 5. linux server ile çalışabilen , 6. yaptığı işlerle ilgili referans gösterebilen, 7. teknik dokümantasyonu takip edebilecek seviyede i̇ngilizce bilen 8. erkek adaylar için askerliğini tamamlamış veya 3 yıl tecilli 9. benzer pozisyonda en az 1 yıl deneyimli, '

# CLEANING STOP WORDS / STEP 4

İlanlar içinde geçen stop wordlerin temizlenmesi.

In [29]:
clean_data_v1 = data_without_special_terms.copy()
clean_data_v1.head()

,Unnamed: 0,PozisyonAdi,metin
0,134,yazılım uzmanı,"dünyada 150 ülkede, 27 farklı dil ile yaklaşı..."
1,137,yazılım uzmanı,"genel ni̇teli̇kler BLLTX üniversitelerin, bil..."
2,270,oyun yazılım uzmanı,oyun geliştirmeyi ve oynamayı hayat felsefesi...
3,586,yazılım uzmanı,şirketimizde görevlendirilmek üzere aşağıdaki...
4,624,yazılım uzmanı,BLLTX yazılım geliştirme projelerinde aktif y...


In [30]:
clean_data_v1 = clean_data_v1.drop(['Unnamed: 0'], axis=1)
clean_data_v1.head()

,PozisyonAdi,metin
0,yazılım uzmanı,"dünyada 150 ülkede, 27 farklı dil ile yaklaşı..."
1,yazılım uzmanı,"genel ni̇teli̇kler BLLTX üniversitelerin, bil..."
2,oyun yazılım uzmanı,oyun geliştirmeyi ve oynamayı hayat felsefesi...
3,yazılım uzmanı,şirketimizde görevlendirilmek üzere aşağıdaki...
4,yazılım uzmanı,BLLTX yazılım geliştirme projelerinde aktif y...


## CREATE ENGLISH AND TURKISH STOP WORDS LIST

In [31]:
english_stop_words = []
turkish_stop_words = []

with open(os.path.abspath('..')+"\Data\_english-stop-words.txt",encoding="utf8") as file:
    for line in file:
        line=line.lower()
        english_stop_words.append(line.strip())
        
with open(os.path.abspath('..')+"\Data\_turkce-stop-words.txt",encoding="utf8") as file:
    for line in file:
        line=line.lower()
        turkish_stop_words.append(line.strip())


## DETERMINE THE LANGUAGE OF THE TEXT

In [32]:
clean_data_v1['Dil']=clean_data_v1["metin"].apply(lambda text: "Türkçe" if " the " not  in  text else ("İngilizce/Türkçe"  if " ve " in text else "İngilizce"))
clean_data_v1.head()

,PozisyonAdi,metin,Dil
0,yazılım uzmanı,"dünyada 150 ülkede, 27 farklı dil ile yaklaşı...",Türkçe
1,yazılım uzmanı,"genel ni̇teli̇kler BLLTX üniversitelerin, bil...",Türkçe
2,oyun yazılım uzmanı,oyun geliştirmeyi ve oynamayı hayat felsefesi...,Türkçe
3,yazılım uzmanı,şirketimizde görevlendirilmek üzere aşağıdaki...,Türkçe
4,yazılım uzmanı,BLLTX yazılım geliştirme projelerinde aktif y...,Türkçe


In [33]:
clean_data_v1["Dil"] .value_counts()

Türkçe              1086
İngilizce             34
İngilizce/Türkçe       3
Name: Dil, dtype: int64

## CLEAN "İngilizce/Türkçe" TEXT

In [34]:
clean_data_v1=clean_data_v1.drop(clean_data_v1.loc[(clean_data_v1['Dil'] == "İngilizce/Türkçe")].index)
clean_data_v1["Dil"].value_counts()

Türkçe       1086
İngilizce      34
Name: Dil, dtype: int64

## CLEAN STOP WORDS

In [35]:
for stop_word in turkish_stop_words:
    clean_data_v1.loc[(clean_data_v1['Dil'] == "Türkçe","metin")] = clean_data_v1["metin"].apply(lambda text: text.replace(" "+stop_word+" "," "))  
    
for stop_word in english_stop_words:
    clean_data_v1.loc[(clean_data_v1['Dil'] == "İngilizce","metin")] = clean_data_v1["metin"].apply(lambda text: text.replace(" "+stop_word+" "," "))  

In [36]:
clean_data_v1.head()

,PozisyonAdi,metin,Dil
0,yazılım uzmanı,"dünyada 150 ülkede, 27 farklı dil yaklaşık 60...",Türkçe
1,yazılım uzmanı,"genel ni̇teli̇kler BLLTX üniversitelerin, bil...",Türkçe
2,oyun yazılım uzmanı,oyun geliştirmeyi oynamayı hayat felsefesine ...,Türkçe
3,yazılım uzmanı,şirketimizde görevlendirilmek aşağıdaki nitel...,Türkçe
4,yazılım uzmanı,BLLTX yazılım geliştirme projelerinde aktif y...,Türkçe


In [37]:
clean_data_v1.to_csv('yazilim_uzmani_data_v1.csv') 

# PARSING / STEP 5

Metinleri madde işareti ve noktaya göre cümlelere ayırma işlemi

In [38]:
clean_data_v2 = pd.read_csv("yazilim_uzmani_data_v1.csv")

In [39]:
clean_data_v2 = clean_data_v2.drop("Unnamed: 0",axis=1)

In [40]:
clean_data_v2.head()

,PozisyonAdi,metin,Dil
0,yazılım uzmanı,"dünyada 150 ülkede, 27 farklı dil yaklaşık 60...",Türkçe
1,yazılım uzmanı,"genel ni̇teli̇kler BLLTX üniversitelerin, bil...",Türkçe
2,oyun yazılım uzmanı,oyun geliştirmeyi oynamayı hayat felsefesine ...,Türkçe
3,yazılım uzmanı,şirketimizde görevlendirilmek aşağıdaki nitel...,Türkçe
4,yazılım uzmanı,BLLTX yazılım geliştirme projelerinde aktif y...,Türkçe


In [41]:
clean_data_v2["metin"][25]

' yazılım ekibimizde görevlendirilmek üzere, ekip çalışmasına yatkın, yeni teknolojilere gelişmeye açık, analitik sorgulama becerisi yüksek yoğun iş temposuna ayak uydurabilecek takım arkadaşları aramaktayız. adaylarda aranan özellikler: - üniversitelerin bilgisayar mühendisliği benzeri 4 yıllık bölümlerinden mezun, - mezuniyet sonrası yazılım uzmanı minimum 4 yıllık iş deneyimi olan, - veritabanı sistemlerine hakim iyi derecede  SPCL261SPCL   SPCL113SPCL  dili bilen , -  SPCL44SPCL ,  SPCL36SPCL   SPCL35SPCL üzerinde tecrübe sahibi, - web tabanlı uygulama geliştirme konusunda deneyimli ( SPCL20SPCL ,  SPCL24SPCL  script), - katmanlı uygulama geliştirme mimarilerinde yazılım geliştirme yapmış, - i̇yi derecede i̇ngilizce bilen. - müşteri ihtiyaçlarını anlayarak yazılım tasarımını oluşturabilecek sorumluluk üstlenebilecek, - erkek adayların askerliğini tamamlamış 2 yıl tecilli gerekmektedir i̇ş tanimi firmamızın yürüttüğü e-devlet projelerinde müşteriye özel çözüm geliştirme projelerinde

## PARSING TEXT ACCORDING TO "BLLTX" AND "."

In [42]:
clean_data_v2["metin"]=clean_data_v2["metin"].apply(lambda text: re.split("BLLTX|\.",text ))

In [43]:
clean_data_v2["metin"][30]

[' innovance, finans sektörüne yönelik yazılım, analiz proje yönetimi konularında danışmanlık hizmeti vermektedir',
 ' danışmanlığını yapmakta olduğumuz banka adına yürüttüğümüz projelerde görevlendirmek "react developer" arayışımız bulunmaktadır',
 ' ',
 ' üniversitelerin 4 yıllık bölümlerinden mezun (bilgisayar, matematik, endüstri, i̇şletme mühendislikleri yönetim bilişim sistemleri (mis) i̇statistik) react',
 "js, react native projelerinde 2 yıl görev almış,  SPCL24SPCL script'e hakim ,  SPCL13SPCL  geliştirme alakalı güncel teknolojileri takip eden, typescript , es6+,  SPCL20SPCL 5,css3 konularında deneyimli, git akışlarında kod geliştirme tecrübesi olan, i̇yi derecede i̇ngilizce bilgisine sahip",
 ' erkek adaylar askerliğini tamamlamış minimum 2 yıl tecilli takım arkadaşları arıyoruz',
 ' ']

## CREATE SENTENCE DF FOR PARSING

In [44]:
sentences = pd.DataFrame(columns=['PostId', 'Sentence', 'Language'])

In [45]:
for i in range(len(clean_data_v2)):
    for j in range (len(clean_data_v2["metin"][i])):
        sentences.loc[len(sentences.index)] = ["Post"+str(i), clean_data_v2["metin"][i][j], clean_data_v2["Dil"][i]] 

sentences.head()    

,PostId,Sentence,Language
0,Post0,"dünyada 150 ülkede, 27 farklı dil yaklaşık 60",Türkçe
1,Post0,000 firma kullanılan erp yazılımı SPCL38SPCL b...,Türkçe
2,Post0,,Türkçe
3,Post0,üniversitelerin lisans bölümlerinden mezun,Türkçe
4,Post0,erp pojelerinde (test uzmanı olarak) 2 yıl te...,Türkçe


In [46]:
sentences.to_csv("yazilim_uzmani_sentences_v1.csv")  

In [47]:
len(sentences)

16067

# CLEAN NEW SENTENCES  DF / STEP 6

Noktalama işaretleri, sayılar, fazla boşluklar ve boş satırların temizlenmesi

## FUNCTION TO CLEAN NUMBERS WITH CONSIDERING MASKING

In [48]:
sentences["Sentence"][145]

' i̇ş tanimi '

In [49]:
def clean_numbers_without_mask(text):
    clean = re.compile('[0-9]+\\b(?!SPCL\\b)+')
    return re.sub(clean, ' ', str(text))
#Maskeler hariç tüm sayıların temizlenmesi

In [50]:
sentences["Sentence"]=sentences['Sentence'].apply(lambda text : clean_numbers_without_mask(text))
sentences.head()

,PostId,Sentence,Language
0,Post0,"dünyada ülkede, farklı dil yaklaşık",Türkçe
1,Post0,firma kullanılan erp yazılımı SPCL38SPCL bus...,Türkçe
2,Post0,,Türkçe
3,Post0,üniversitelerin lisans bölümlerinden mezun,Türkçe
4,Post0,erp pojelerinde (test uzmanı olarak) yıl te...,Türkçe


In [51]:
sentences["Sentence"][145]

' i̇ş tanimi '

## FUNCTION TO CLEAN PUNITIONS (Without ",")

In [52]:
def clean_punitions(text):
    clean = re.compile('[\/#!$%\^&\*\?;:{}=\-_`~()]|’[^\s]+|\'[^\s]+|\’[^\s]+|’\s[^\s]+')
    return re.sub(clean, ' ', str(text))
#' işaretlerinden sonra ki ek de temizkendi. Öğreneğin :  2003’ te türkiye'nin ilk savunma yazılımı i̇hracatını yaptık

In [53]:
sentences["Sentence"]=sentences['Sentence'].apply(lambda text : clean_punitions(text))
sentences.head()

,PostId,Sentence,Language
0,Post0,"dünyada ülkede, farklı dil yaklaşık",Türkçe
1,Post0,firma kullanılan erp yazılımı SPCL38SPCL bus...,Türkçe
2,Post0,,Türkçe
3,Post0,üniversitelerin lisans bölümlerinden mezun,Türkçe
4,Post0,erp pojelerinde test uzmanı olarak yıl te...,Türkçe


In [54]:
sentences["Sentence"][145]

' i̇ş tanimi '

## CLEANING EXTRA SPACES AND NEW LINES

In [55]:
sentences["Sentence"]=sentences["Sentence"].replace(to_replace=["\n"],value=" ",regex=True,inplace=False)
sentences["Sentence"] = sentences["Sentence"].apply(lambda text: text.strip())
sentences["Sentence"]=sentences['Sentence'].apply(lambda text : re.sub(' +', ' ', text))

In [56]:
sentences["Sentence"][145]

'i̇ş tanimi'

## CLEANING EMPTY LINES

In [57]:
len(sentences)

16067

In [58]:
sentences.drop(sentences[sentences['Sentence'] == "nan" ].index, inplace = True)

In [59]:
len(sentences)

16067

In [60]:
sentences.drop(sentences[sentences['Sentence'] == "" ].index, inplace = True)
sentences.reset_index(inplace=True)

In [61]:
len(sentences)

14271

In [62]:
sentences.drop(['index'],axis=1,inplace=True)

In [63]:
sentences.head(10)

,PostId,Sentence,Language
0,Post0,"dünyada ülkede, farklı dil yaklaşık",Türkçe
1,Post0,firma kullanılan erp yazılımı SPCL38SPCL busin...,Türkçe
2,Post0,üniversitelerin lisans bölümlerinden mezun,Türkçe
3,Post0,erp pojelerinde test uzmanı olarak yıl tecrübeli,Türkçe
4,Post0,analitik matematiksel düşünme yönü kuvvetli,Türkçe
5,Post0,detay odaklı çalışan,Türkçe
6,Post0,raporlama yapan,Türkçe
7,Post0,ekip çalışmasına uyumlu,Türkçe
8,Post0,problem çözme yeteneğine sahip,Türkçe
9,Post0,i̇ş teslim tarihlerine uyan i̇ş tanimi,Türkçe


In [64]:
sentences["Sentence"].head(10)

0                  dünyada ülkede, farklı dil yaklaşık
1    firma kullanılan erp yazılımı SPCL38SPCL busin...
2           üniversitelerin lisans bölümlerinden mezun
3     erp pojelerinde test uzmanı olarak yıl tecrübeli
4          analitik matematiksel düşünme yönü kuvvetli
5                                 detay odaklı çalışan
6                                      raporlama yapan
7                              ekip çalışmasına uyumlu
8                       problem çözme yeteneğine sahip
9               i̇ş teslim tarihlerine uyan i̇ş tanimi
Name: Sentence, dtype: object

In [65]:
sentences.to_csv("yazilim_uzmani_sentences_v2.csv")